In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import tensorflow_addons as tfa
import keras
import torchtext

from sklearn.preprocessing import MultiLabelBinarizer
seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras

import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from torch.nn import BCEWithLogitsLoss
import torch
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, AdamW
from tensorflow_addons.metrics import HammingLoss
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import hamming_loss

In [ ]:
train_sub2_df = pd.read_csv('/content/train_subtask_2.csv')

print(train_sub2_df.shape)
train_sub2_df.head()

dev_sub2_df = pd.read_csv('/content/dev_subtask_2.csv')

print(dev_sub2_df.shape)
dev_sub2_df.head()

test_sub2_df = pd.read_csv('/content/test_subtask_2.csv')
print(test_sub2_df.shape)
test_sub2_df.head()


(433, 10)
(83, 10)
(54, 10)


,Unnamed: 0,id,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,0,3121,The Pope Says 'Humanity Must Repent For Abusin...,We humans “must repent and modify our lifestyl...,the pope say humanity must repent for abuse mo...,"[(the, 'DT'), (pope, 'NN'), (say, 'VBP'), (hum...",[],we human must repent and modify our lifestyle ...,"[(we, 'PRP'), (human, 'NN'), (must, 'MD'), (re...","[(thursday, 'DATE', 391), (the world day, 'DAT..."
1,1,3135,Russian allies are unknowingly working against...,The Secretary of State for Defence described P...,russian ally be unknowingly work against they ...,"[(russian, 'JJ'), (ally, 'NN'), (be, 'VB'), (u...","[(russian, 'NORP', 381)]",the secretary of state for defence describe pu...,"[(the, 'DT'), (secretary, 'NNP'), (of, 'IN'), ...","[(putin, 'PERSON', 380), (russian, 'NORP', 381..."
2,2,3134,"I know this week has been disruptive, says LIZ...",The United Kingdom is the greatest country on ...,know this week have be disruptive say liz truss,"[(know, 'VB'), (this, 'DT'), (week, 'NN'), (ha...","[(this week, 'DATE', 391)]",the united kingdom be the great country on ear...,"[(the, 'DT'), (united, 'NNP'), (kingdom, 'NNP'...","[(the united kingdom, 'GPE', 384), (70 year, '..."
3,3,3120,Vladimir Putin facing Ukraine 'humiliation' as...,"VLADIMIR PUTIN is facing ""humiliation"" in Ukra...",vladimir putin face ukraine humiliation as rus...,"[(vladimir, 'NNP'), (putin, 'NNP'), (face, 'NN...","[(vladimir putin, 'PERSON', 380), (ukraine, 'G...",vladimir putin be face humiliation in ukraine ...,"[(vladimir, 'NNP'), (putin, 'NNP'), (be, 'VB')...","[(vladimir putin, 'PERSON', 380), (ukraine, 'G..."
4,4,3136,What Are the Odds Putin is Bluffing About Usin...,Eurointelligence founder Wolfgang Münchau has ...,what be the odd putin be bluff about use nucle...,"[(what, 'WP'), (be, 'VB'), (the, 'DT'), (odd, ...",[],eurointelligence founder wolfgang münchau have...,"[(eurointelligence, 'NN'), (founder, 'NN'), (w...","[(wolfgang münchau, 'PERSON', 380), (putin, 'P..."


In [ ]:
train_sub2_df["genre"] = train_sub2_df["genre"].str.split(',')
dev_sub2_df["genre"] = dev_sub2_df["genre"].str.split(',')

In [ ]:
train_sub2_df["genre"]

0      [Political, Morality, Fairness_and_equality, E...
1      [Morality, Security_and_defense, Policy_prescr...
2      [Political, Crime_and_punishment, External_reg...
3      [Political, Crime_and_punishment, Fairness_and...
4      [Political, Morality, Fairness_and_equality, E...
                             ...                        
428    [Political, Crime_and_punishment, External_reg...
429    [Political, Morality, Crime_and_punishment, Qu...
430    [Crime_and_punishment, Political, Legality_Con...
431    [Political, Crime_and_punishment, Fairness_and...
432    [Crime_and_punishment, Political, Legality_Con...
Name: genre, Length: 433, dtype: object

In [ ]:
dev_sub2_df["genre"]

0     [Political, Fairness_and_equality, Policy_pres...
1     [Political, Capacity_and_resources, Policy_pre...
2     [Public_opinion, Policy_prescription_and_evalu...
3     [Political, External_regulation_and_reputation...
4     [Public_opinion, Political, External_regulatio...
                            ...                        
78    [Political, Crime_and_punishment, Fairness_and...
79    [Policy_prescription_and_evaluation, Political...
80    [Policy_prescription_and_evaluation, Political...
81    [Crime_and_punishment, Political, Legality_Con...
82    [Political, Morality, Crime_and_punishment, Qu...
Name: genre, Length: 83, dtype: object

In [ ]:
mlb = MultiLabelBinarizer()
multi_train_1 = pd.DataFrame(mlb.fit_transform(train_sub2_df['genre']), columns = mlb.classes_)
multi_train = mlb.fit_transform(train_sub2_df['genre'])

mlb = MultiLabelBinarizer()
multi_dev_1 = pd.DataFrame(mlb.fit_transform(dev_sub2_df['genre']), columns = mlb.classes_)
multi_dev = mlb.fit_transform(dev_sub2_df['genre'])

In [ ]:
multi_train_1

,Capacity_and_resources,Crime_and_punishment,Cultural_identity,Economic,External_regulation_and_reputation,Fairness_and_equality,Health_and_safety,Legality_Constitutionality_and_jurisprudence,Morality,Policy_prescription_and_evaluation,Political,Public_opinion,Quality_of_life,Security_and_defense
0,0,0,0,0,1,1,0,1,1,0,1,0,0,0
1,0,0,0,1,0,0,0,1,1,1,0,0,0,1
2,0,1,0,0,1,0,0,1,0,1,1,1,0,0
3,0,1,0,0,1,1,0,1,0,0,1,0,0,0
4,0,0,0,1,1,1,0,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0,1,0,1,1,0,0,1,0,0,1,0,0,0
429,0,1,0,0,0,0,0,0,1,1,1,0,1,0
430,0,1,0,0,1,0,0,1,0,0,1,0,0,0
431,0,1,0,0,1,1,0,1,0,0,1,0,0,0


In [ ]:
multi_dev_1

,Capacity_and_resources,Crime_and_punishment,Cultural_identity,Economic,External_regulation_and_reputation,Fairness_and_equality,Health_and_safety,Legality_Constitutionality_and_jurisprudence,Morality,Policy_prescription_and_evaluation,Political,Public_opinion,Quality_of_life,Security_and_defense
0,0,0,0,1,0,1,0,0,0,1,1,1,0,1
1,1,0,0,0,0,0,1,1,0,1,1,1,0,1
2,0,0,0,0,1,0,0,0,0,1,1,1,0,0
3,0,0,0,1,1,0,0,1,0,1,1,1,0,0
4,1,0,0,0,1,0,0,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,1,0,0,1,1,0,1,0,1,1,0,0,0
79,0,0,0,0,1,0,0,1,0,1,1,0,0,0
80,0,0,0,0,1,0,0,1,0,1,1,0,0,0
81,0,1,0,0,1,0,0,1,0,0,1,0,0,0


In [ ]:
multi_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
type(multi_train)

numpy.ndarray

In [ ]:
multi_dev

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0]])

In [ ]:
# aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", aug_max=4)

In [ ]:
x_train = train_sub2_df.preprocessed_articles + train_sub2_df.preprocessed_headlines
y_train = multi_train

x_dev = dev_sub2_df.preprocessed_articles + dev_sub2_df.preprocessed_headlines
y_dev = multi_dev

# x_train = train_sub1_df.text
# y_train = train_sub1_df.target

x_test = test_sub2_df.preprocessed_articles + test_sub2_df.preprocessed_headlines

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', num_labels=14, output_attentions=True)

In [ ]:
x_train

0      far leave comedienne chelsea handler have admi...
1      those be the time the time page be of january ...
2      carload of crazy head to the white house want ...
3      but of course this make no difference to the p...
4      this weekend in manchester england yellow vest...
                             ...                        
428    on wednesday donald trump be fly to military t...
429    the mueller circus cost the taxpayer million b...
430    special counsel robert mueller will not recomm...
431    the democratic presidential contender be unifo...
432    no new indictment will come from special couns...
Length: 433, dtype: object

In [ ]:
x_dev

0     theresa may could lose the united kingdom over...
1     if britain leave the european union without tr...
2     with three month until britain leave the europ...
3     the british parliament just hand prime ministe...
4     as british mp be tell that their february brea...
                            ...                        
78    hungarian prime minister viktor orban have be ...
79    deep state strike back and miss good oman for ...
80    jeremy hunt insist theresa may can still salva...
81    mueller conclude russia trump probe lawmaker u...
82    tudor simionov be kill when he try to prevent ...
Length: 83, dtype: object

In [ ]:
augmented_sentences=[]
augmented_sentences_labels=[]
count = 0

for i in x_train.index:
    count+=1
    if count%10==0:
      print("Running count = ", count)
    try:
      temps=aug.augment(x_train[i], n=3)
      for sent in temps:
        augmented_sentences.append(sent)
        augmented_sentences_labels.append(y_train[i])
    except:
      continue

Running count =  10
Running count =  20
Running count =  30
Running count =  40
Running count =  50
Running count =  60
Running count =  70
Running count =  80
Running count =  90
Running count =  100
Running count =  110
Running count =  120
Running count =  130
Running count =  140
Running count =  150
Running count =  160
Running count =  170
Running count =  180
Running count =  190
Running count =  200
Running count =  210
Running count =  220
Running count =  230
Running count =  240
Running count =  250
Running count =  260
Running count =  270
Running count =  280
Running count =  290
Running count =  300
Running count =  310
Running count =  320
Running count =  330
Running count =  340
Running count =  350
Running count =  360
Running count =  370
Running count =  380
Running count =  390
Running count =  400
Running count =  410
Running count =  420
Running count =  430


In [ ]:
x_train_aug = pd.Series()
y_train_aug = pd.Series()
x_train_aug = x_train_aug.append(pd.Series(augmented_sentences), ignore_index=True)
y_train_aug = y_train_aug.append(pd.Series(augmented_sentences_labels), ignore_index=False)

In [ ]:
df=pd.concat([x_train_aug,y_train_aug],axis=1)

In [ ]:
df.columns =['x_train', 'y_train']

In [ ]:
df

,x_train,y_train
0,far leave comedienne chelsea handler have admi...,"[0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0]"
1,far leave comedienne chelsea handler have admi...,"[0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0]"
2,far leave comedienne chelsea handler have admi...,"[0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0]"
3,those be the time the time page be of january ...,"[0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1]"
4,those be the time the time page be of january ...,"[0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1]"
...,...,...
1294,the democratic presidential contender be unifo...,"[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]"
1295,the next presidential contender be uniformly c...,"[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]"
1296,the new indictment will come from special coun...,"[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]"
1297,no new indictment will come from special couns...,"[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]"


In [ ]:
df.to_csv('aug.csv')

In [ ]:
x_train_aug.shape

(1299,)

In [ ]:
y_train_aug.shape

(1299,)

In [ ]:
type(y_train_aug)

pandas.core.series.Series

In [ ]:
arr = np.stack(y_train_aug.to_numpy())
arr.shape
# np_arr

(1299, 14)

In [ ]:
y_train_aug_1 = arr

In [ ]:
y_train_aug_1.shape

(1299, 14)

In [ ]:
x_train = x_train_aug.to_numpy().reshape(-1)

x_dev = x_dev.to_numpy().reshape(-1)

In [ ]:
x_train.shape

(1299,)

In [ ]:
def roberta_encode(texts, tokenizer):
    MAX_LEN = 512
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(str(text))
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
x_train = roberta_encode(x_train, tokenizer)
x_dev = roberta_encode(x_dev, tokenizer)
x_test = roberta_encode(x_test, tokenizer)

y_train = np.asarray(y_train_aug_1, dtype='int32')
y_dev = np.asarray(y_dev, dtype='int32')

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
def build_model(n_categories):
    MAX_LEN = 512
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

    # Import RoBERTa model from HuggingFace
    roberta_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
    # custom_objects = {"TFRobertaForSequenceClassification": TFRobertaForSequenceClassification}
    # config = roberta_model.get_config()
    # with tf.keras.utils.custom_object_scope(custom_objects):
    #     roberta_model = TFRobertaForSequenceClassification.from_config(config)
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, embeddings are the first one,
    # so let's slice out the first position
    x = x[0]

    x = tf.keras.layers.Dropout(rate=0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                  loss='binary_crossentropy', 
                  metrics=[tf.keras.metrics.BinaryAccuracy(),
                           tfa.metrics.F1Score(num_classes=14,
                                               average='micro', threshold=0.07142857)])
    
    return model

In [ ]:
x_test['input_type_ids'].shape

(54, 512)

In [ ]:
model = build_model(14)
model.summary()

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 512)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_roberta_for_sequence_classi  TFSequenceClassifie  124647170  ['input_word_ids[0][0]',         
 fication (TFRobertaForSequence  rOutput(loss=None,               'input_mask[0][0]',         

In [ ]:
x_train['input_word_ids'].shape

(1299, 512)

In [ ]:
x_dev['input_word_ids'].shape

(83, 512)

In [ ]:
y_dev.shape

(83, 14)

In [ ]:
y_train.shape

(1299, 14)

In [ ]:
y_dev.shape

(83, 14)

In [ ]:
metrics=[tf.keras.metrics.BinaryAccuracy(),
                           tfa.metrics.F1Score(num_classes=14,
                                               average='micro', threshold=0.50)]

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = '/content/Modelcheckpoints/Checkpoint-{epoch:02d}.h5'

checkpoint = ModelCheckpoint(filepath,monitor= tfa.metrics.F1Score(num_classes=14,
                                               average='micro', threshold=0.50), varbose=1, save_best_only=True, mode='max')

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=8, validation_data=(x_dev, y_dev), callbacks=[checkpoint])

Epoch 1/100
163/163 [==============================] - ETA: 0s - loss: 0.3114 - binary_accuracy: 0.8422 - f1_score: 0.6285

163/163 [==============================] - 21s 132ms/step - loss: 0.3114 - binary_accuracy: 0.8422 - f1_score: 0.6285 - val_loss: 0.5775 - val_binary_accuracy: 0.7702 - val_f1_score: 0.6438
Epoch 2/100
163/163 [==============================] - ETA: 0s - loss: 0.3114 - binary_accuracy: 0.8418 - f1_score: 0.6276

163/163 [==============================] - 21s 131ms/step - loss: 0.3114 - binary_accuracy: 0.8418 - f1_score: 0.6276 - val_loss: 0.5790 - val_binary_accuracy: 0.7685 - val_f1_score: 0.6402
Epoch 3/100
163/163 [==============================] - ETA: 0s - loss: 0.3147 - binary_accuracy: 0.8416 - f1_score: 0.6264

163/163 [==============================] - 21s 131ms/step - loss: 0.3147 - binary_accuracy: 0.8416 - f1_score: 0.6264 - val_loss: 0.5577 - val_binary_accuracy: 0.7642 - val_f1_score: 0.6395
Epoch 4/100
163/163 [==============================] - ETA: 0s - loss: 0.3165 - binary_accuracy: 0.8381 - f1_score: 0.6256

163/163 [==============================] - 21s 131ms/step - loss: 0.3165 - binary_accuracy: 0.8381 - f1_score: 0.6256 - val_loss: 0.5557 - val_binary_accuracy: 0.7754 - val_f1_score: 0.6401
Epoch 5/100
163/163 [==============================] - ETA: 0s - loss: 0.3164 - binary_accuracy: 0.8391 - f1_score: 0.6251

...

163/163 [==============================] - 21s 131ms/step - loss: 0.3111 - binary_accuracy: 0.8443 - f1_score: 0.6281 - val_loss: 0.5936 - val_binary_accuracy: 0.7668 - val_f1_score: 0.6334
Epoch 97/100
163/163 [==============================] - ETA: 0s - loss: 0.3079 - binary_accuracy: 0.8447 - f1_score: 0.6309

163/163 [==============================] - 21s 131ms/step - loss: 0.3079 - binary_accuracy: 0.8447 - f1_score: 0.6309 - val_loss: 0.5675 - val_binary_accuracy: 0.7478 - val_f1_score: 0.6204
Epoch 98/100
163/163 [==============================] - ETA: 0s - loss: 0.3091 - binary_accuracy: 0.8431 - f1_score: 0.6290

163/163 [==============================] - 21s 131ms/step - loss: 0.3091 - binary_accuracy: 0.8431 - f1_score: 0.6290 - val_loss: 0.6071 - val_binary_accuracy: 0.7694 - val_f1_score: 0.6369
Epoch 99/100
163/163 [==============================] - ETA: 0s - loss: 0.3075 - binary_accuracy: 0.8475 - f1_score: 0.6293

163/163 [==============================] - 21s 131ms/step - loss: 0.3075 - binary_accuracy: 0.8475 - f1_score: 0.6293 - val_loss: 0.5789 - val_binary_accuracy: 0.7668 - val_f1_score: 0.6444
Epoch 100/100
163/163 [==============================] - ETA: 0s - loss: 0.3046 - binary_accuracy: 0.8458 - f1_score: 0.6313

163/163 [==============================] - 21s 131ms/step - loss: 0.3046 - binary_accuracy: 0.8458 - f1_score: 0.6313 - val_loss: 0.6248 - val_binary_accuracy: 0.7616 - val_f1_score: 0.6447


In [ ]:
model.save('saved_model_50.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/saved_model.h5', custom_objects={"TFRobertaForSequenceClassification": TFRobertaForSequenceClassification})


In [ ]:
y_pred = model.predict(x_dev)

3/3 [==============================] - 0s 129ms/step


In [ ]:
y_pred

array([[4.9344415e-04, 4.6960655e-01, 1.8979996e-07, ..., 2.8424831e-02,
        1.9806236e-02, 4.8562530e-01],
       [1.2835817e-03, 4.9289480e-01, 9.2147599e-07, ..., 4.2553972e-02,
        2.9222982e-02, 4.7528741e-01],
       [4.0068164e-05, 4.0298054e-01, 5.6325620e-09, ..., 1.0107853e-02,
        8.9798514e-03, 5.3466773e-01],
       ...,
       [4.1233809e-03, 5.1636583e-01, 8.0409054e-06, ..., 7.2636984e-02,
        5.0171807e-02, 4.6538296e-01],
       [5.0681097e-05, 3.8879842e-01, 4.4955406e-04, ..., 1.9753436e-03,
        1.2927170e-01, 2.2906853e-01],
       [1.3335099e-06, 9.8672420e-01, 3.0047055e-05, ..., 2.2469736e-03,
        3.4595937e-02, 7.4994850e-01]], dtype=float32)

In [ ]:
from sklearn.metrics import f1_score

y_pred_final = list()
y_pred_sub = list()

for i in range(0,100,1):
  print(i/100)
  y_pred_final = tf.cast(y_pred >= (i/100), tf.int32)
  print('Macro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='macro')))
  print('Micro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='micro')))

0.0
Macro f1_score = 0.4524247922807242
Micro f1_score = 0.5309734513274336
0.01
Macro f1_score = 0.5043211244783301
Micro f1_score = 0.6314948041566747
0.02
Macro f1_score = 0.5091027061387937
Micro f1_score = 0.6367346938775511
0.03
Macro f1_score = 0.5151689669659337
Micro f1_score = 0.6411960132890365
0.04
Macro f1_score = 0.5181379960419925
Micro f1_score = 0.6439522998296423
0.05
Macro f1_score = 0.5146247407645486
Micro f1_score = 0.6458874458874458
0.06
Macro f1_score = 0.5111276004990918
Micro f1_score = 0.647887323943662
0.07
Macro f1_score = 0.5065576482796497
Micro f1_score = 0.6441281138790035
0.08
Macro f1_score = 0.5057202400295929
Micro f1_score = 0.6450450450450451
0.09
Macro f1_score = 0.5023243749576414
Micro f1_score = 0.6459854014598541
0.1
Macro f1_score = 0.4975563138989924
Micro f1_score = 0.6457564575645757
0.11
Macro f1_score = 0.49182131972936716
Micro f1_score = 0.6492048643592143
0.12
Macro f1_score = 0.4929748411810122
Micro f1_score = 0.6502835538752363
0

In [ ]:
from sklearn.metrics import f1_score

y_pred_final = list()
y_pred_sub = list()

y_pred_final = tf.cast(y_pred >= 0.245, tf.int32)

In [ ]:
from sklearn.metrics import f1_score
print('Macro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='macro')))
print('Micro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='micro')))

Macro f1_score = 0.43090712332829256
Micro f1_score = 0.6673532440782698


In [ ]:
y_test_pred = model.predict(x_test)

2/2 [==============================] - 0s 109ms/step


In [ ]:
y_test_pred

array([[3.78544927e-02, 4.32862871e-04, 1.92104205e-01, 5.67054230e-05,
        5.46836033e-02, 5.92186349e-03, 8.48935485e-01, 6.55928180e-02,
        1.59151211e-01, 3.04280955e-04, 8.07181075e-02, 2.33302108e-05,
        8.16562057e-01, 3.43438059e-01],
       [1.04765745e-03, 4.36005414e-01, 1.62145487e-04, 2.41903514e-01,
        6.93706512e-01, 2.05634311e-01, 1.57296844e-03, 4.22028393e-01,
        3.43572348e-01, 1.33375451e-01, 9.96014118e-01, 5.65001853e-02,
        4.83288765e-02, 5.13999462e-01],
       [1.70431986e-01, 5.19380808e-01, 8.62299502e-02, 3.45427543e-01,
        5.01037478e-01, 3.45825583e-01, 1.82836384e-01, 5.61769903e-01,
        4.47155386e-01, 4.47027713e-01, 7.22581267e-01, 3.41911465e-01,
        3.12203288e-01, 5.00406563e-01],
       [3.47395241e-02, 5.18322766e-01, 6.88113086e-03, 2.60332316e-01,
        5.34751832e-01, 2.35445291e-01, 4.06732261e-02, 5.66639006e-01,
        4.00513053e-01, 3.63725781e-01, 9.12174225e-01, 2.03001440e-01,
        1.600

In [ ]:
y_test_final = list()

y_test_final = tf.cast(y_test_pred >= 0.245, tf.int32)

In [ ]:
px = pd.DataFrame(y_test_final.numpy())

In [ ]:
px

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0,0,0,0,0,1,0,0,0,0,0,1,1
1,0,1,0,0,1,0,0,1,1,0,1,0,0,1
2,0,1,0,1,1,1,0,1,1,1,1,1,1,1
3,0,1,0,1,1,0,0,1,1,1,1,0,0,1
4,0,1,0,0,1,0,0,1,1,0,1,0,0,1
5,0,0,0,0,0,0,1,0,0,0,0,0,1,1
6,0,1,0,0,1,0,0,1,1,0,1,0,0,1
7,0,1,0,0,1,1,1,1,1,1,1,1,1,1
8,0,1,0,0,1,0,0,1,0,0,1,0,0,1
9,0,1,0,0,1,0,0,1,1,1,1,0,0,1


In [ ]:
px.to_csv('submission_roberta_50_6.csv')